---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [100]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # import .txt file but must include the 'sep='\n'' statement to properly display the data
    u_towns = pd.read_csv('university_towns.txt', sep='\n', header=None, names=['RegionName'])
    # create a 'State' column where the state names are those found in the 'Region Name' column having the word 'edit'
    u_towns['State'] = np.where(u_towns['RegionName'].str.contains('edit'), u_towns['RegionName'],np.NaN)
    # create a new dataframe with the specified data
    u_towns = u_towns[['State', 'RegionName']]
    # using '.str.replace', remove the '[]' and the characters inside it 
    u_towns['State'] = u_towns['State'].str.replace(r'\[.*\]','')
    # remove '.str.strip', remove the whitespaces
    u_towns['State'] = u_towns['State'].str.strip()
    # using '.str.replace', remove the '[]' and '()' and the characters inside them
    u_towns['RegionName'] = u_towns['RegionName'].str.replace(r'\[.*\]','').str.replace(r'\(.*\)','')
    # remove '.str.strip', remove the whitespaces
    u_towns['RegionName'] = u_towns['RegionName'].str.strip()
    # fill the NaN spaces from the 'State' column
    u_towns['State'].fillna(method='ffill', inplace=True)
    # remove the rows with identical 'State' and 'Region Name' values
    u_towns = u_towns[u_towns['State'] != u_towns['RegionName']]
    
    return u_towns
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [12]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # import excel file and prepare data by removing some unneeded rows
    GDP = pd.read_excel('gdplev.xls', skiprows=5)
    # using the '.iloc' function filter the data by removing unneeded rows and columns 
    GDP = GDP.iloc[2:,[4,6]]
    # rename the name of the column headers
    GDP.columns = ['Quarter', 'GDP']
    # create a new column with the year as the data
    GDP['Year'] = GDP['Quarter'].str[0:4].astype(int)
    # filter the data by only including those of year 2000 and onward
    GDP = GDP[GDP['Year']>=2000]
    # remove the year column
    GDP = GDP.iloc[:,0:2]
    # create a new column which contains the difference between the current and previous GDP
    GDP['Diff'] = GDP['GDP'].diff()
    # create a new column which contains the original index of the data
    # note: this step can be skipped but during reset_index, it should be 'drop=False', and a column for the original index will be created
    GDP['Index'] = GDP.index
    # reset the index
    GDP.reset_index(drop=True, inplace=True)
    
    # filter the data by only including rows with a negative 'Diff' or a drop in GDP
    GDP_drop = GDP[GDP['Diff'] < 0]
    # create a new column which contains the difference between the current and previous 'Index'
    GDP_drop['Index Diff'] = GDP_drop['Index'].diff()
    # the index of the smallest 'Index Diff' is taken
    rec_start_index = GDP_drop['Index Diff'].idxmin()
    # using the '.iloc' function, the quarter of the start of the recession is acquired 
    rec_start = GDP['Quarter'].iloc[rec_start_index-1]

    return rec_start
get_recession_start()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'2008q3'

In [13]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # import excel file and prepare data by removing some unneeded rows
    GDP = pd.read_excel('gdplev.xls', skiprows=5)
    # using the '.iloc' function filter the data by removing unneeded rows and columns 
    GDP = GDP.iloc[2:,[4,6]]
    # rename the name of the column headers
    GDP.columns = ['Quarter', 'GDP']
    # create a new column with the year as the data
    GDP['Year'] = GDP['Quarter'].str[0:4].astype(int)
    # filter the data by only including those of year 2000 and onward
    GDP = GDP[GDP['Year']>=2000]
    # remove the year column
    GDP = GDP.iloc[:,0:2]
    # create a new column which contains the difference between the current and previous GDP
    GDP['Diff'] = GDP['GDP'].diff()
    # create a new column which contains the original index of the data
    # note: this step can be skipped but during reset_index, it should be 'drop=False', and a column for the original index will be created
    GDP['Index'] = GDP.index
    # reset the index
    GDP.reset_index(drop=True, inplace=True)
       
    # filter the data by only including rows with a positive 'Diff' or a gain in GDP
    GDP_up = GDP[GDP['Diff'] > 0]
    # create a new column which contains the difference between the current and previous 'Index'
    GDP_up['Index Diff'] = GDP_up['Index'].diff()
    # the index of the largest 'Index Diff is taken'
    rec_end_index = GDP_up['Index Diff'].idxmax()
    # using the '.iloc' function, the quarter of the end of the recession is acquired 
    rec_end = GDP['Quarter'].iloc[rec_end_index+1]

    return rec_end
get_recession_end()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'2009q4'

In [14]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # import excel file and prepare data by removing some unneeded rows
    GDP = pd.read_excel('gdplev.xls', skiprows=5)
    # using the '.iloc' function filter the data by removing unneeded rows and columns 
    GDP = GDP.iloc[2:,[4,6]]
    # rename the name of the column headers
    GDP.columns = ['Quarter', 'GDP']
    # create a new column with the year as the data
    GDP['Year'] = GDP['Quarter'].str[0:4].astype(int)
    # filter the data by only including those of year 2000 and onward
    GDP = GDP[GDP['Year']>=2000]
    # remove the year column
    GDP = GDP.iloc[:,0:2]
    # create a new column which contains the difference between the current and previous GDP
    GDP['Diff'] = GDP['GDP'].diff()
    # create a new column which contains the original index of the data
    # note: this step can be skipped but during reset_index, it should be 'drop=False', and a column for the original index will be created
    GDP['Index'] = GDP.index
    # reset the index
    GDP.reset_index(drop=True, inplace=True)
    
    # acquire the index of the start of recession
    index_rec_start = GDP[GDP['Quarter'] == get_recession_start()].index[0]
    # acquire the index of the end of recession
    index_rec_end = GDP[GDP['Quarter'] == get_recession_end()].index[0]
    # create a dataframe which includes only the rows from index_rec_start to index_rec_end
    GDP_rec = GDP.iloc[index_rec_start: index_rec_end,:]
    # take the quarter of the bottom or the one having the least GDP
    bottom_quarter = GDP['Quarter'].iloc[GDP_rec['GDP'].idxmin()]
    return bottom_quarter
get_recession_bottom()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'2009q2'

In [64]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # upload the dat file
    city_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    # get the column # for the '1996-04' column name
    remove_start = city_homes.columns.get_loc('1996-04')
    # get the column # for the '2000-01' column name
    remove_end = city_homes.columns.get_loc('2000-01')
    # filter the data by removing all year columns prior the year 2000
    city_homes.drop(city_homes.columns[remove_start: remove_end], axis=1, inplace=True)
    # filter the data by removing the unneeded columns
    city_homes.drop(city_homes.columns[[0,3,4,5]], axis=1, inplace=True)
    # using the '.map' function, the 'States' of each region is updated to a full word
    city_homes['State'] = city_homes['State'].map(states)
    
    # create a dataframe with multiindex of 'State' and 'RegionName'
    city_homes = city_homes.set_index(['State','RegionName']).sort_index()
    # convert the column names to datetime format
    city_homes.columns = pd.to_datetime(city_homes.columns)
    # clump the header names together in quarters and take the mean for each quarter
    city_homes = city_homes.resample('Q',axis=1).mean()
    # format the column names so that they will be in per 'Quarter' period
    city_homes.columns = city_homes.columns.to_period('Q').strftime('%Yq%q')
    return city_homes
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [115]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # assign the result from housing data to a variable
    housing_data = convert_housing_data_to_quarters()
    # get the index of start of the recession
    rec_start = (housing_data.columns.get_loc(get_recession_start())-1)
    # get the index of bottom of the recession
    rec_bot = (housing_data.columns.get_loc(get_recession_bottom()))
    # assign the result from university town data to a variable and make the 'State' and 'RegionName' as indices
    uni = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    # create a new column which is quotient of the GDP during the start and GDP during the bottom of the recession
    # *a value greater than 1 means the GDP dropped during the recession period
    # using '.to_frame', convert the series into a dataframe 
    # using '.dropna', filter the data by removing all empty rows
    housing_data = (housing_data.iloc[:, rec_start] / housing_data.iloc[:, rec_bot]).to_frame().dropna()
    
    # create a dataframe by merge the university town data with the housing data by their indices
    uni_df = uni.merge(housing_data, how='inner', left_index=True, right_index=True)
    # create a dataframe which is the non-intersect of uni_df dataframe
    nonuni_df = housing_data.drop(uni_df.index)
    
    # run the ttest to compute for the p_value
    p_value = ttest_ind(uni_df.values, nonuni_df.values).pvalue
    
    # create an 'if statement' to tell if there is a significant difference in p_value (1% confidence level)
    if p_value < 0.01:
        different = True
    else:
        different = False
    
    # create an 'if statement' to tell where is the GDP better during the recession
    if uni_df.mean().values < nonuni_df.mean().values:
        better = 'university town'
    else:
        better = 'non-university town'
    
    return (different, p_value[0], better)

In [116]:
run_ttest()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(True, 0.0013238023200683538, 'university town')